In [1]:
# Se realiza el proceso de ETL para ello se importa estas librerías
import json
import pandas as pd
import ast
import numpy as np

In [3]:
# Se crea el DataFrame
data = []
ruta_archivo = r'..\Archivos Json\australian_user_reviews.json'
with open(ruta_archivo, 'rb') as f:  # Aquí se corrige el error
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))

user_reviews = pd.DataFrame(data)

In [4]:
# Se verifica el dataframe creado
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
# Se oberva que tipos de datos hay
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [6]:
# Se divide el dataframe para evitar que se crashee el equipo
chunks = np.array_split(user_reviews, 100)

c:\Users\Paola\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
# Se realiza un 1er desanidado

# Inicializar una lista para almacenar los resultados desanidados
desanidado_list = []

# Iterar sobre cada parte, desanidarla y agregarla a la lista de resultados
for chunk in chunks:
    desanidado_chunk = chunk.explode('reviews')
    desanidado_list.append(desanidado_chunk)

In [8]:
# Se concatena en un nuevo dataframe
user_reviews1 = pd.concat(desanidado_list, ignore_index=True)

In [9]:
# Se oberva el nuevo dataframe creado
user_reviews1

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [10]:
# Se revisa que datos hay en la columna 'reviews'
user_reviews1.loc[1, 'reviews']

{'funny': '',
 'posted': 'Posted July 15, 2011.',
 'last_edited': '',
 'item_id': '22200',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': "It's unique and worth a playthrough."}

In [11]:
# Se vuelve a desanidar y dividir el dataframe
chunks = np.array_split(user_reviews1, 110)

c:\Users\Paola\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
# Inicializar una lista para almacenar los resultados desanidados
desanidado_list = []

# Iterar sobre cada parte, desanidarla y agregarla a la lista de resultados
for chunk in chunks:
    desanidado_chunk = chunk['reviews'].apply(pd.Series)
    desanidado_chunk['user_id'] = chunk['user_id']
    desanidado_chunk['user_url'] = chunk['user_url']
    desanidado_list.append(desanidado_chunk)

In [13]:
# Concatenar los resultados desanidados en un último DataFrame
user_reviews2 = pd.concat(desanidado_list, ignore_index=True)

In [14]:
# Nueva verificacion del dataframe
user_reviews2

,funny,posted,last_edited,item_id,helpful,recommend,review,0,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59331,,Posted July 20.,,730,No ratings yet,True,:D,NaN,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [15]:
# Se verifica los tipos de datos de cada columna
user_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   funny        59305 non-null  object 
 1   posted       59305 non-null  object 
 2   last_edited  59305 non-null  object 
 3   item_id      59305 non-null  object 
 4   helpful      59305 non-null  object 
 5   recommend    59305 non-null  object 
 6   review       59305 non-null  object 
 7   0            0 non-null      float64
 8   user_id      59333 non-null  object 
 9   user_url     59333 non-null  object 
dtypes: float64(1), object(9)
memory usage: 4.5+ MB


In [16]:
# Se imprime los nombres de las columnas
print(user_reviews2.columns)

Index([      'funny',      'posted', 'last_edited',     'item_id',
           'helpful',   'recommend',      'review',             0,
           'user_id',    'user_url'],
      dtype='object')


In [17]:
# Se elimina la columna 0 por tener todos los valores nulos
user_reviews2 = user_reviews2.drop(0, axis=1)

In [18]:
# Una revision mas del dataframe
user_reviews2.tail(50)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
59283,,"Posted October 10, 2015.",,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...,76561198251004808,http://steamcommunity.com/profiles/76561198251...
59284,,Posted February 12.,,730,No ratings yet,True,Great competitive game! Good for anyone that l...,76561198251328528,http://steamcommunity.com/profiles/76561198251...
59285,,Posted March 13.,,369580,No ratings yet,True,i guees it was pretty lols,76561198254993106,http://steamcommunity.com/profiles/76561198254...
59286,,Posted March 12.,,353560,No ratings yet,True,GET IT SCRUB!!!!!!!!!!!!!,76561198254993106,http://steamcommunity.com/profiles/76561198254...
59287,,Posted March 2.,,440,No ratings yet,True,its mlg,76561198254993106,http://steamcommunity.com/profiles/76561198254...
59288,,Posted May 9.,,224480,1 of 1 people (100%) found this review helpful,True,Very fun game. very rich story line which flow...,kimjongadam,http://steamcommunity.com/id/kimjongadam
59289,,Posted March 30.,,314160,No ratings yet,True,"FSX = BaeMust buy!!!Also, if you have a couple...",kimjongadam,http://steamcommunity.com/id/kimjongadam
59290,,Posted March 26.,,273110,No ratings yet,False,Well there is no other review I can give to th...,kimjongadam,http://steamcommunity.com/id/kimjongadam
59291,,Posted March 26.,,264200,No ratings yet,True,One finger death punch is Bae. It's simplistic...,kimjongadam,http://steamcommunity.com/id/kimjongadam
59292,,Posted February 26.,,239820,No ratings yet,True,"I love game dev tycoon, you can create dream g...",kimjongadam,http://steamcommunity.com/id/kimjongadam


In [19]:
# Se guarda el archivo ya transformado a un formato csv
ruta_guardado_csv = "../Datacsv/user_reviews.csv"
user_reviews2.to_csv(ruta_guardado_csv, index=False)